In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
import os
import json
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import JSONLoader
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
import time
import pandas as pd

# 设置OpenAI API密钥
os.environ["OPENAI_API_KEY"] = ""

# 加载CSV文件
csv_file_path = 'ptt_stock_articles.csv'
comments_file_path = 'ptt_stock_comments.csv'

df = pd.read_csv(csv_file_path)
df_comments = pd.read_csv(comments_file_path)

# 将CSV文件转换为JSONL文件
jsonl_file_path = 'combined_ptt_stock_documents.jsonl'
df.to_json(jsonl_file_path, orient='records', lines=True, force_ascii=False)

# 加载JSONL文件
loader = JSONLoader(
    file_path=jsonl_file_path,
    jq_schema='.Content',
    json_lines=True,
    text_content=True
)

# 初始化OpenAI Embeddings
embeddings = OpenAIEmbeddings()

# 加载文档
documents_load = loader.load()

# 初始化Chroma并将文档添加到Chroma
chroma_client = chromadb.Client(Settings())

# 检查集合是否存在
collection_name = 'recommend_collection'
collection_list = chroma_client.list_collections()

if collection_name in [col.name for col in collection_list]:
    print(f"Collection {collection_name} already exists. Deleting the existing collection.")
    chroma_client.delete_collection(collection_name)

# 创建集合
collection = chroma_client.create_collection(
    name=collection_name,
    embedding_function=embedding_functions.OpenAIEmbeddingFunction(
        api_key=os.environ['OPENAI_API_KEY'],
        model_name="text-embedding-ada-002"
    )
)

# 處理文章並將它們添加到Chroma
def process_documents_and_add_to_chroma(documents, collection, batch_size=100, delay=60):
    total_docs = len(documents)
    for start_idx in range(0, total_docs, batch_size):
        end_idx = min(start_idx + batch_size, total_docs)
        batch = documents[start_idx:end_idx]
        document_ids = [f"doc_{i}" for i in range(start_idx, end_idx)]  # 生成文档ID列表
        batch_contents = [doc.page_content[:1000] for doc in batch]  # 提取文档内容并限制长度为1000字符
        
        try:
            # 使用 embed_documents 方法來獲取批量文檔的嵌入
            batch_embeddings = embeddings.embed_documents(batch_contents)
            
            # 确保嵌入和ID数量一致
            if len(batch_embeddings) != len(batch_contents):
                raise ValueError(f"Number of embeddings ({len(batch_embeddings)}) must match number of documents ({len(batch_contents)})")
            
            # 確保 ID 數量與文檔数量一致
            if len(document_ids) != len(batch_contents):
                raise ValueError(f"Number of document IDs ({len(document_ids)}) must match number of documents ({len(batch_contents)})")
            
            collection.add(documents=batch_contents, embeddings=batch_embeddings, ids=document_ids)
            
        except Exception as e:
            print(f"Error processing batch from index {start_idx} to {end_idx}: {e}")
        
        if end_idx < total_docs:
            print(f"Batch {start_idx//batch_size + 1} completed. Waiting for {delay} seconds...")
            time.sleep(delay)

# 處理文檔並添加到Chroma
process_documents_and_add_to_chroma(documents_load, collection, batch_size=100, delay=60)

# 定义新的股票情绪分析prompt模板
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
query = "請問川普對台股有什麼影響？"
user_question = query

template = """你是一個股票分析助手，根據使用者提供的問題以及PTT上的文章和留言進行股票情緒分析。

PTT上的文章和留言: {background_context}

Hard requirements and steps that must be followed:
1. 請描述使用者提供的問題。
2. 使用提供的PTT資料，分析台股情绪。
3. 請列出文章標題和主要內容。
4. 請總結每篇文章的情绪（正面、負面、中立）。
5. 如果有不一致的評價，請詳細說明。
6. 請使用簡明扼要的語言回答。

用詞要有禮貌與專業！"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{user_question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 初始化ChatOpenAI
chat_model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 遍歷前幾篇文章的標題和内容，并生成回應
try:
    responses = []
    
    for idx, doc in enumerate(documents_load[:3]):  # 只处理前幾篇文章
        title = doc.metadata.get('Title', 'No Title')  # 確保文檔中有標題
        content = doc.page_content[:1000]  # 限制内容长度为1000字符
        
        # 生成回應
        response = chat_model(chat_prompt.format_messages(background_context=[content], user_question=user_question))
        responses.append(f"Title: {title}\nContent: {content}\nResponse: {response.content}\n\n")
        print(response)
    # 将所有回應保存到文本文件
    with open("generated_responses_test.txt", "w", encoding="utf-8") as f:
        f.writelines(responses)
        
except Exception as e:
    response = f"Error during document processing or response generation: {e}"
    print(response)
    with open("generated_responses_test.txt", "w", encoding="utf-8") as f:
        f.write(response)
    
print(response)